In [24]:
import numpy as np
import matplotlib
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import csv
import json
import os

In [25]:
imagefolder = '/u/y/u/yuhanl/Downloads/NextGenMaskRCNN-master/code/datasets/ellipse/Data3TypesYminXminYmaxXmax5/images'
imagename = "0501_300kx_1nm_clhaadf3_0014.jpg"

In [26]:
from collections import defaultdict
All_Image_Defects_List = list()
columns = defaultdict(list) # each value in each column is appended to a list
current_img_defect_List = list()

with open('/u/y/u/yuhanl/Downloads/0501_300kx_1nm_clhaadf3_0014_results.csv') as f:
    reader = csv.DictReader(f) # read rows into a dictionary format
    for row in reader: # read a row as {column1: value1, column2: value2,...}
        for (k,v) in row.items(): # go over each column name and value 
            columns[k].append(v) # append the value into the appropriate list
                                 # based on column name k
x = columns['X'] #the central point coordinates
y = columns['Y']
a = columns['Major']
b = columns['Minor']
angle = columns['Angle']
All_Image_Defects_List = list()
current_img_defect_List = list()
defects_Dict = dict()
defects_X_List = list()
defects_Y_List = list()
for k in range(len(columns)):
    xk = float(x[k])
    yk = float(y[k])
    ak = float(a[k]) 
    bk = float(b[k]) 
    anglek = float(angle[k]) * (-1)
    defects_Dict = dict()
    defects_X_List = list()
    defects_Y_List = list()
    ellipse = Ellipse([xk,yk],ak,bk,anglek)
    vertices = ellipse.get_verts()
    
    x_coor = []
    y_coor = []
    for i in range(len(vertices)):
        x_coor.append(vertices[i][0])
        y_coor.append(vertices[i][1])

    for i in range(len(vertices)):
        defects_X_List.append(x_coor[i])
        defects_Y_List.append(y_coor[i])
        
    defects_Dict['X'] = defects_X_List
    defects_Dict['Y'] = defects_Y_List
    current_img_defect_List.append(defects_Dict)

All_Image_Defects_List.append(current_img_defect_List)

In [27]:
with open("via_region_data.json","w+") as jsonfile:
    with open("/u/y/u/yuhanl/Downloads/NextGenMaskRCNN-master/code/datasets/ellipse/Data3TypesYminXminYmaxXmax5/trainimages.txt") as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    each_file = dict()
    for i in range(len(All_Image_Defects_List)):
        img_i = All_Image_Defects_List[i]
        file_size = os.path.getsize(imagefolder + '/'+content[i])
        filename_size = content[i] + str(file_size)
        each_file[filename_size] = {'fileref':"",'size':file_size,'filename':content[i]}
        each_file[filename_size].update({'base64_img_data':"",'file_attributes':{},'regions':{}})
        each_bbox = each_file[filename_size]['regions']
        for j in range(len(img_i)):
            each_bbox.update({str(j):{'shape_attributes':{},'region_attributes':{}}})
            each_bbox[str(j)]['shape_attributes'] = {'name':"polygon"}
            each_bbox[str(j)]['shape_attributes'].update({'all_points_x':[],'all_points_y':[]})
            each_bbox[str(j)]['shape_attributes']['all_points_x'] = np.asarray(img_i[j]['X']).tolist()
            each_bbox[str(j)]['shape_attributes']['all_points_y'] = np.asarray(img_i[j]['Y']).tolist()

    json.dump(each_file,jsonfile)